In [1]:
import pandas as pd 
import fastai

df = pd.read_csv("acl2kpm250pm_auto300.csv")
df_test = pd.read_csv("background_first_sent2.csv")[200:]
df_test = df_test.dropna()

fastai.__version__

df["text"] = df["text"].map(replace_outof_vocab_words)
df_test["text"] = df_test["text"].map(replace_outof_vocab_words)

ModuleNotFoundError: No module named 'fastai'

In [2]:
from fastai.text.all import *

dls = TextDataLoaders.from_df(df, text_col='text', label_col='labels')
dls.show_batch()

learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, FBeta(beta=1)])
learn.fine_tune(4, 1e-2)
learn.show_results()
learn.save("model_3")

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5)  
interp.plot_confusion_matrix()

learn.predict("We discuss the implications of a more 'civic-minded journalism.")

ModuleNotFoundError: No module named 'fastai'